In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
# Chargement des données depuis le fichier CSV
df_filtered = pd.read_csv('../data_processed/df_north_vac.csv', parse_dates=['Date_et_heure_de_comptage'])
df_filtered['heure'] = pd.to_datetime(df_filtered['heure'], format='%H:%M:%S').dt.hour
df_dummies = pd.get_dummies(df_filtered['C_AR'], prefix='arrondissement')
df_processed = pd.concat([df_filtered, df_dummies], axis=1)
X = df_processed.drop(['Identifiant_arc', 'Libelle', 'Date_et_heure_de_comptage', 'Etat_trafic', 'C_AR'], axis=1)
y = df_processed['Etat_trafic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Calcul de l'accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Affichage du rapport de classification
report = classification_report(y_test, y_pred)
print(f"Classification Report:\n{report}")


In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Chargement des données
df = pd.read_csv('../data_processed/df_filtered_north_zone.csv', parse_dates=['Date_et_heure_de_comptage'])
vacances_df = pd.read_csv('../data_processed/vacances.csv', parse_dates=['start_date', 'end_date'])

# Ajouter une colonne 'vacances' au dataframe principal
df['vacances'] = df['Date_et_heure_de_comptage'].apply(
    lambda x: any((vacances_df['start_date'] <= x) & (x <= vacances_df['end_date']))
).astype(int)

# Extraire le jour de la semaine et le numéro de la semaine
df['jour_semaine'] = df['Date_et_heure_de_comptage'].dt.dayofweek
df['semaine_annee'] = df['Date_et_heure_de_comptage'].dt.isocalendar().week

# Convertir la colonne 'Etat_trafic' en labels numériques
df['Etat_trafic'] = df['Etat_trafic'].astype('category').cat.codes

# Sélectionner les caractéristiques et la variable cible
X = df[['jour_semaine', 'semaine_annee', 'vacances']]
y = df['Etat_trafic']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardiser les caractéristiques
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Entraîner un modèle de régression logistique
model = LogisticRegression()
model.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluer le modèle
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


/var/folders/l7/rlzcchm55wsby2638rc0pfdh0000gn/T/ipykernel_23807/3369517148.py:9: DtypeWarning: Columns (13,14,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data_processed/df_filtered_north_zone.csv', parse_dates=['Date_et_heure_de_comptage'])


[[     0   6718      0      0]
 [     0 173159      0      0]
 [     0  34397      0      0]
 [     0  16840      0      0]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      6718
           1       0.75      1.00      0.86    173159
           2       0.00      0.00      0.00     34397
           3       0.00      0.00      0.00     16840

    accuracy                           0.75    231114
   macro avg       0.19      0.25      0.21    231114
weighted avg       0.56      0.75      0.64    231114



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample

# Chargement des données avec des types spécifiques
df = pd.read_csv('../data_processed/df_filtered_north_zone.csv', parse_dates=['Date_et_heure_de_comptage'], dtype={
    'Identifiant_arc': 'int',
    'Libelle': 'str',
    'Débit_horaire': 'float',
    "Taux_d'occupation": 'float',
    'Etat_trafic': 'str',
    'C_AR': 'int',
    'annee': 'int',
    'mois': 'int',
    'jour': 'int',
    'heure': 'str'
})

# Chargement des données de vacances
vacances_df = pd.read_csv('../data_processed/vacances.csv', parse_dates=['start_date', 'end_date'])

# Ajouter une colonne 'vacances' au dataframe principal
df['vacances'] = df['Date_et_heure_de_comptage'].apply(
    lambda x: any((vacances_df['start_date'] <= x) & (x <= vacances_df['end_date']))
).astype(int)

# Extraire le jour de la semaine et le numéro de la semaine
df['jour_semaine'] = df['Date_et_heure_de_comptage'].dt.dayofweek
df['semaine_annee'] = df['Date_et_heure_de_comptage'].dt.isocalendar().week

# Convertir la colonne 'Etat_trafic' en labels numériques
df['Etat_trafic'] = df['Etat_trafic'].astype('category').cat.codes

# Sélectionner les caractéristiques et la variable cible
X = df[['jour_semaine', 'semaine_annee', 'vacances']]
y = df['Etat_trafic']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Combiner X_train et y_train pour le rééquilibrage
train_data = pd.concat([X_train, y_train], axis=1)

# Rééquilibrer les classes par suréchantillonnage des classes minoritaires
resampled_data = []
for cls in train_data['Etat_trafic'].value_counts().index:
    class_sample = train_data[train_data['Etat_trafic'] == cls]
    resampled_class = resample(class_sample, 
                               replace=True, 
                               n_samples=train_data['Etat_trafic'].value_counts().max(), 
                               random_state=42)
    resampled_data.append(resampled_class)
resampled_data = pd.concat(resampled_data)

# Séparer les caractéristiques et la variable cible après rééquilibrage
X_train_resampled = resampled_data.drop('Etat_trafic', axis=1)
y_train_resampled = resampled_data['Etat_trafic']

# Standardiser les caractéristiques
scaler = StandardScaler()
X_train_resampled = scaler.fit_transform(X_train_resampled)
X_test = scaler.transform(X_test)

# Entraîner un modèle de régression logistique
model = LogisticRegression()
model.fit(X_train_resampled, y_train_resampled)

# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluer le modèle
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Vérifier l'équilibrage des classes après suréchantillonnage
print(resampled_data['Etat_trafic'].value_counts())


/var/folders/l7/rlzcchm55wsby2638rc0pfdh0000gn/T/ipykernel_23807/1186323886.py:9: DtypeWarning: Columns (13,14,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data_processed/df_filtered_north_zone.csv', parse_dates=['Date_et_heure_de_comptage'], dtype={


[[ 3041  2222   236  1219]
 [58407 82703  7129 24920]
 [13829 13291  1331  5946]
 [ 7020  5976   576  3268]]
              precision    recall  f1-score   support

           0       0.04      0.45      0.07      6718
           1       0.79      0.48      0.60    173159
           2       0.14      0.04      0.06     34397
           3       0.09      0.19      0.13     16840

    accuracy                           0.39    231114
   macro avg       0.27      0.29      0.21    231114
weighted avg       0.62      0.39      0.47    231114

Etat_trafic
1    692474
2    692474
3    692474
0    692474
Name: count, dtype: int64


In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample

# Chargement des données avec des types spécifiques
df = pd.read_csv('../data_processed/df_filtered_north_zone.csv', parse_dates=['Date_et_heure_de_comptage'], dtype={
    'Identifiant_arc': 'int',
    'Libelle': 'str',
    'Débit_horaire': 'float',
    "Taux_d'occupation": 'float',
    'Etat_trafic': 'str',
    'C_AR': 'int',
    'annee': 'int',
    'mois': 'int',
    'jour': 'int',
    'heure': 'str'
})

# Chargement des données de vacances
vacances_df = pd.read_csv('../data_processed/vacances.csv', parse_dates=['start_date', 'end_date'])

# Ajouter une colonne 'vacances' au dataframe principal
df['vacances'] = df['Date_et_heure_de_comptage'].apply(
    lambda x: any((vacances_df['start_date'] <= x) & (x <= vacances_df['end_date']))
).astype(int)

# Extraire le jour de la semaine et le numéro de la semaine
df['jour_semaine'] = df['Date_et_heure_de_comptage'].dt.dayofweek
df['semaine_annee'] = df['Date_et_heure_de_comptage'].dt.isocalendar().week

# Convertir la colonne 'Etat_trafic' en labels numériques
df['Etat_trafic'] = df['Etat_trafic'].astype('category').cat.codes

# Sélectionner les caractéristiques et la variable cible
X = df[['jour_semaine', 'semaine_annee', 'vacances']]
y = df['Etat_trafic']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Combiner X_train et y_train pour le rééquilibrage
train_data = pd.concat([X_train, y_train], axis=1)

# Rééquilibrer les classes par suréchantillonnage des classes minoritaires
resampled_data = []
for cls in train_data['Etat_trafic'].value_counts().index:
    class_sample = train_data[train_data['Etat_trafic'] == cls]
    resampled_class = resample(class_sample, 
                               replace=True, 
                               n_samples=train_data['Etat_trafic'].value_counts().max(), 
                               random_state=42)
    resampled_data.append(resampled_class)
resampled_data = pd.concat(resampled_data)

# Séparer les caractéristiques et la variable cible après rééquilibrage
X_train_resampled = resampled_data.drop('Etat_trafic', axis=1)
y_train_resampled = resampled_data['Etat_trafic']

# Standardiser les caractéristiques
scaler = StandardScaler()
X_train_resampled = scaler.fit_transform(X_train_resampled)
X_test = scaler.transform(X_test)

# Entraîner un modèle de Random Forest
model = RandomForestClassifier(random_state=42)
model.fit(X_train_resampled, y_train_resampled)

# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluer le modèle
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Vérifier l'équilibrage des classes après suréchantillonnage
print(resampled_data['Etat_trafic'].value_counts())


/var/folders/l7/rlzcchm55wsby2638rc0pfdh0000gn/T/ipykernel_23807/3817860737.py:9: DtypeWarning: Columns (13,14,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data_processed/df_filtered_north_zone.csv', parse_dates=['Date_et_heure_de_comptage'], dtype={


[[ 3376  1196   895  1251]
 [51941 69487 24654 27077]
 [12733  8898  5958  6808]
 [ 6805  3678  2470  3887]]
              precision    recall  f1-score   support

           0       0.05      0.50      0.08      6718
           1       0.83      0.40      0.54    173159
           2       0.18      0.17      0.17     34397
           3       0.10      0.23      0.14     16840

    accuracy                           0.36    231114
   macro avg       0.29      0.33      0.23    231114
weighted avg       0.66      0.36      0.44    231114

Etat_trafic
1    692474
2    692474
3    692474
0    692474
Name: count, dtype: int64


In [41]:
pip install holidays

  Obtaining dependency information for holidays from https://files.pythonhosted.org/packages/7b/ac/034c669a23438b6d5a2d42d3d73ddad8439fc8b7057a884e4639cb2698f9/holidays-0.52-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample
from datetime import datetime
import holidays

# Charger les données
df = pd.read_csv('../data_processed/df_filtered_north_zone.csv', parse_dates=['Date_et_heure_de_comptage'], dtype={
    'Identifiant_arc': 'int',
    'Libelle': 'str',
    'Débit_horaire': 'float',
    "Taux_d'occupation": 'float',
    'Etat_trafic': 'str',
    'C_AR': 'int',
    'annee': 'int',
    'mois': 'int',
    'jour': 'int',
    'heure': 'str'
})

vacances_df = pd.read_csv('../data_processed/vacances.csv', parse_dates=['start_date', 'end_date'])

# Ajouter une colonne 'vacances' au dataframe principal
df['vacances'] = df['Date_et_heure_de_comptage'].apply(
    lambda x: any((vacances_df['start_date'] <= x) & (x <= vacances_df['end_date']))
).astype(int)

# Ajouter une colonne 'jours_feries' au dataframe principal
fr_holidays = holidays.France()
df['jours_feries'] = df['Date_et_heure_de_comptage'].apply(
    lambda x: 1 if x in fr_holidays else 0
)

# Extraire le jour de la semaine et le numéro de la semaine
df['jour_semaine'] = df['Date_et_heure_de_comptage'].dt.dayofweek
df['semaine_annee'] = df['Date_et_heure_de_comptage'].dt.isocalendar().week

# Convertir la colonne 'Etat_trafic' en labels numériques
df['Etat_trafic'] = df['Etat_trafic'].astype('category').cat.codes

# Sélectionner les caractéristiques et la variable cible
X = df[['jour_semaine', 'semaine_annee', 'vacances', 'jours_feries']]
y = df['Etat_trafic']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Combiner X_train et y_train pour le rééquilibrage
train_data = pd.concat([X_train, y_train], axis=1)

# Rééquilibrer les classes par suréchantillonnage des classes minoritaires
resampled_data = []
for cls in train_data['Etat_trafic'].value_counts().index:
    class_sample = train_data[train_data['Etat_trafic'] == cls]
    resampled_class = resample(class_sample, 
                               replace=True, 
                               n_samples=train_data['Etat_trafic'].value_counts().max(), 
                               random_state=42)
    resampled_data.append(resampled_class)
resampled_data = pd.concat(resampled_data)

# Séparer les caractéristiques et la variable cible après rééquilibrage
X_train_resampled = resampled_data.drop('Etat_trafic', axis=1)
y_train_resampled = resampled_data['Etat_trafic']

# Standardiser les caractéristiques
scaler = StandardScaler()
X_train_resampled = scaler.fit_transform(X_train_resampled)
X_test = scaler.transform(X_test)

# Paramètres pour l'optimisation du modèle Random Forest
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Recherche des meilleurs paramètres
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(X_train_resampled, y_train_resampled)

# Meilleur modèle
best_model = grid_search.best_estimator_

# Faire des pr


/var/folders/l7/rlzcchm55wsby2638rc0pfdh0000gn/T/ipykernel_23807/2889537326.py:12: DtypeWarning: Columns (13,14,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data_processed/df_filtered_north_zone.csv', parse_dates=['Date_et_heure_de_comptage'], dtype={


KeyboardInterrupt: 